In [33]:
from py2neo import Graph
from py2neo import Node, Relationship
import pandas as pd
import numpy as np

g = Graph()
g.delete_all()

In [34]:
df = pd.read_csv('./AtgO2.txt', sep="\t", names=["target", "source", "type"])

nodes = []

s = df["source"].astype(str)
t = df["target"].astype(str)

all_nodes = s.append(t).unique()
print(len(all_nodes))

712


In [36]:
res = g.size
print(res)

# Assign genes
df2 = pd.read_csv('./AtgO2_terms_to_genes.txt', sep="\t", names=["term", "genes"])

assigned_genes = {}

for row in df2.itertuples():
    genes = row.genes
    gene_list = genes.split('|')
    assigned_genes[str(row.term)] = gene_list


0


In [38]:
nodes = {}

for row in df.itertuples():
    source_name = str(row.source)
    target_name = str(row.target)
    
    if source_name in nodes:
        source = nodes[source_name]
    else:
        if source_name in assigned_genes:
            source = Node("term", name=source_name, assigned_genes=assigned_genes[source_name])
        else:
            source = Node("term", name=source_name)
        
        nodes[source_name] = source
    
    if target_name in nodes:
        target = nodes[target_name]
    else:
        if target_name in assigned_genes:
            target = Node("term", name=target_name, assigned_genes=assigned_genes[target_name])
        else:
            target = Node("term", name=target_name)
        nodes[target_name] = target
    
    edge = Relationship(source, "child_of", target)
    g.create(edge)

In [27]:
a = g.cypher.execute("MATCH (n:Person) RETURN n")
print(a)



SyntaxException: expected START or CREATE
"MATCH (n:Person) RETURN n"
 ^

In [15]:
for row in df2.itertuples():
    genes = row.genes
    gene_list = genes.split('|')
    print(len(gene_list))

492
484
476
384
156
322
238
139
211
46
139
80
102
73
42
19
10
24
22
18
5
4
12
12
17
9
8
9
9
10
7
8
8
8
8
5
4
8
6
8
6
6
5
5
8
5
5
5
5
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
8
3
3
3
3
3
3
3
3
3
3
3
2
3
3
3
3
4
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
4
4
5
5
5
6
6
8
9
9
11
14
17
19
20
23
23
24
28
30
41
44
46
300
431
444


In [ ]:
for line in raw_interactions:
  new_line = line.rstrip()

  vals = new_line.split("\t")
  source = g.get_indexed_node("Vertex", "name", vals[0])
  target = g.get_indexed_node("Vertex", "name", vals[2])
  if source is not None and target is not None:
    newEdge = g.create(rel(source,"raw_interaction",target))

raw_interactions.close()

print(g.size())